In [1]:
from mtsoo import *
from slgep_lib.chromosome import *
import numpy as np
from copy import deepcopy


In [2]:
names = ['CartPole-v1','custom_gym:cartpole-half-gravity-v0']
envs = GymTaskSet(names)
envs.envs

[<TimeLimit<CartPoleEnv<CartPole-v1>>>,

In [3]:
callback = None
K = len(envs.envs)                 # number of function
N = 4 * K         # population size
T = 100            # number of iteration
mr = 0.1
rmp = 0.3
no_par = 3

max_arity = 2
h_main = 7
h_adf = 5
no_main = envs.envs[0].action_space.n
no_adf = no_main*2
no_terminal = envs.envs[0].reset().shape[0]

In [4]:
population = Slgep_pop(no_adf, no_terminal, no_main,h_main, max_arity, h_adf, no_pop=2*N, no_task=K)
D = population.pop[0].D
D

74

In [5]:
population.evaluate(envs)

In [6]:
for i in range(2*N):
    print(population.pop[i].scalar_fitness)

0.16666666666666666
0.3333333333333333
0.2
1.0
0.5
1.0
0.3333333333333333
0.25
0.16666666666666666
0.2
0.125
0.1111111111111111
0.14285714285714285
0.14285714285714285
0.25
0.5


In [7]:
for i in range(2*N):
    print(population.pop[i].factorial_cost)

[ inf -10.]
[ inf -16.]
[-10.  inf]
[-14.  inf]
[ inf -17.]
[ inf -58.]
[-12.  inf]
[-11.  inf]
[-10.  inf]
[ inf -14.]
[inf -9.]
[inf -9.]
[-10.  inf]
[ inf -10.]
[ inf -16.]
[-13.  inf]


In [8]:
population.sort()

In [9]:
t=0

In [10]:
for k in range(0,N, no_par):
    no_p = np.min((N-k,no_par))
    # learn rmp
    subpops = population.get_subpops()
    rmp_matrix = population.learn_rmp(subpops)
    bl = np.random.normal(0.7, 0.1, size=(no_p, D))

    parents = [population.pop[k+j] for j in range(no_p)]
    same_sf = True
    for i in range(no_p-1):
        if parents[i].sf != parents[i+1].sf:
            same_sf = False
    
    if same_sf:
        cl = population.crossover_mul(parents,bl)
        for i in range(no_p):
            cl[i] = population.mutate(cl[i], mr)
            cl[i].sf = parents[0].sf
    elif np.random.rand()<rmp:
        cl = population.crossover_mul(parents,bl)
        for i in range(no_p):
            cl[i] = population.mutate(cl[i], mr)

            # assign random skill factor from parents to child
            sf_assign = [p.sf for p in parents]
            cl[i].sf = np.random.choice(sf_assign)

In [11]:
cl = population.crossover_mul(parents,bl)

In [24]:
for i in range(no_p):
    cl[i] = population.mutate(cl[i], mr)
    sf_assign = [p.sf for p in parents]
    cl[i].sf = np.random.choice(sf_assign)

In [22]:
cl[0].gene

array([ 6,  5,  4,  6,  6,  5,  4, 14, 16, 17, 15, 15, 16, 17, 15, 11,  5,
        5,  5,  4,  7,  6, 16, 15, 16, 16, 16, 16, 15, 16,  5,  4,  3,  4,
        6, 13, 13, 13, 13, 12, 13,  4,  6,  2,  7,  4, 13, 13, 13, 12, 12,
       12,  2,  0,  5,  6,  2, 13, 13, 13, 13, 13, 13,  3,  4,  1,  3,  1,
       12, 13, 13, 12, 13, 13], dtype=int32)

In [ ]:
     # crossover
    if sf1 == sf2:
        c1, c2 = population.onepoint_crossover(p1, p2)
        c1 = population.mutate(c1, mr)
        c2 = population.mutate(c2, mr)
        # c1, c2 = variable_swap(c1, c2, pswap)
        c1.sf = sf1
        c2.sf = sf1
    elif sf1 != sf2 and np.random.rand() < rmp:
        c1, c2 = population.onepoint_crossover(p1, p2)
        c1 = population.mutate(c1, mr)
        c2 = population.mutate(c2, mr)
        # c1, c2 = variable_swap(c1, c2, pswap)
        if np.random.rand() < 0.5:
            c1.sf = sf1
        else:
            c1.sf = sf2
        if np.random.rand() < 0.5:
            c2.sf = sf1
        else:
            c2.sf = sf2
    else:
        p2 = population.find_relative(sf1)

        c1, c2 = population.onepoint_crossover(p1, p2)
        c1 = population.mutate(c1, mr)
        c2 = population.mutate(c2, mr)
        # c1, c2 = variable_swap(c1, c2, pswap)
        c1.sf = sf1
        c2.sf = sf1

    # replace parent with child
    population.pop[N+i], population.pop[N+i+1] = c1, c2
population.evaluate(envs)
population.sort()
xx ,b0 = population._get_best_individual_of_task(0)
yy, b1 = population._get_best_individual_of_task(1)
print(b0, b1)

In [29]:
for i in range(2*N):
    print(population.pop[i].factorial_cost)

[-77. -10.]
[  inf -247.]
[ inf -55.]
[-77.  inf]
[ inf -41.]
[-77. -38.]
[-46.  inf]
[-12. -41.]
[-46.  -9.]
[-38.  inf]
[ inf -32.]
[ inf -17.]
[-34.  inf]
[ inf -16.]
[-9. inf]
[inf -9.]


In [20]:
for i in range(2*N):
    print(population.pop[i].sf)

1
1
1
0
1
1
0
0
1
0
1
1
0
1
0
1


In [21]:
for i in range(2*N):
    print(population.pop[i].scalar_fitness)

1.0
1.0
0.5
0.5
0.3333333333333333
0.3333333333333333
0.25
0.25
0.2
0.16666666666666666
0.16666666666666666
0.14285714285714285
0.14285714285714285
0.125
0.1111111111111111
0.1


In [26]:
pops = [p for p in population.pop if p.sf == 1]
pops[0].factorial_cost

array([-77., -10.])

In [33]:
agent = pops[0]
sf = agent.sf
result = envs.run_env(sf, agent)
result

-10.0

In [ ]:
fc = [p.factorial_cost[1]]

In [39]:
no_pop = len(population.pop)
fc = []
for i in range(no_pop):
    agent = population.pop[i]
    sf = agent.sf
    result = envs.run_env(sf, agent)
    agent.factorial_cost[sf] = result
    fc.append(agent.factorial_cost)
fc

[array([-77., -10.]),
 array([  inf, -247.]),
 array([ inf, -55.]),
 array([-77.,  inf]),
 array([ inf, -41.]),
 array([-77., -38.]),
 array([-46.,  inf]),
 array([-12., -41.]),
 array([-46.,  -9.]),
 array([-38.,  inf]),
 array([ inf, -32.]),
 array([ inf, -17.]),
 array([-34.,  inf]),
 array([ inf, -16.]),
 array([-9., inf]),
 array([inf, -9.])]

In [40]:
ranking = np.argsort(np.argsort(fc, axis=0), axis=0) + 1
ranking

array([[ 1,  9],
       [10,  1],
       [11,  2],
       [ 2, 12],
       [12,  3],
       [ 3,  5],
       [ 4, 13],
       [ 8,  4],
       [ 5, 10],
       [ 6, 14],
       [13,  6],
       [14,  7],
       [ 7, 15],
       [15,  8],
       [ 9, 16],
       [16, 11]])

In [42]:
np.argmin(ranking, axis=1)

array([0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

In [43]:
1 / np.min(ranking, axis=1)

array([1.        , 1.        , 0.5       , 0.5       , 0.33333333,
       0.33333333, 0.25      , 0.25      , 0.2       , 0.16666667,
       0.16666667, 0.14285714, 0.14285714, 0.125     , 0.11111111,
       0.09090909])

In [ ]:
a